# Pruebas del Sistema de Inventario

Este notebook contiene pruebas para validar el funcionamiento del sistema de altas y bajas de inventario.

In [ ]:
import requests
import json

# Configuración base
BASE_URL = 'http://localhost:3000/api'
headers = {
    'Content-Type': 'application/json'
}

## 1. Login como administrador
Primero necesitamos obtener un token de acceso

In [ ]:
def login_admin():
    login_data = {
        'username': 'admin',  # Reemplaza con tu usuario admin
        'password': 'tu_password'  # Reemplaza con tu contraseña
    }
    
    response = requests.post(f'{BASE_URL}/auth/login', json=login_data)
    if response.status_code == 200:
        token = response.json()['token']
        return {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
    else:
        raise Exception('Error en login:', response.json())

# Obtener headers con token
headers_auth = login_admin()

## 2. Prueba de Alta de Inventario
Crearemos una solicitud de entrada de inventario

In [ ]:
def crear_solicitud_entrada():
    data_entrada = {
        "tipoMovimiento": "ENTRADA",
        "inventarioTipo": "INTERIOR",
        "itemId": "ID_DEL_ITEM",  # Reemplazar con un ID válido
        "cantidad": 5,
        "motivoSolicitud": "Prueba de entrada de inventario",
        "numerosSerie": ["TEST001", "TEST002", "TEST003", "TEST004", "TEST005"]
    }
    
    response = requests.post(
        f'{BASE_URL}/inventory-requests',
        headers=headers_auth,
        json=data_entrada
    )
    
    print('Respuesta:', json.dumps(response.json(), indent=2))
    return response.json()

solicitud_entrada = crear_solicitud_entrada()

## 3. Verificar el estado del inventario antes de aprobar

In [ ]:
def verificar_item(item_id):
    response = requests.get(
        f'{BASE_URL}/inventory/items/{item_id}',
        headers=headers_auth
    )
    print('Estado del item:', json.dumps(response.json(), indent=2))
    return response.json()

estado_inicial = verificar_item(solicitud_entrada['request']['itemId'])

## 4. Aprobar la solicitud de entrada

In [ ]:
def aprobar_solicitud(request_id):
    data_aprobacion = {
        "action": "APROBAR"
    }
    
    response = requests.put(
        f'{BASE_URL}/inventory-requests/{request_id}/process',
        headers=headers_auth,
        json=data_aprobacion
    )
    
    print('Respuesta de aprobación:', json.dumps(response.json(), indent=2))
    return response.json()

aprobacion = aprobar_solicitud(solicitud_entrada['request']['_id'])

## 5. Verificar el estado final del inventario

In [ ]:
estado_final = verificar_item(solicitud_entrada['request']['itemId'])

# Validar que la cantidad se incrementó correctamente
print(f"\nCantidad inicial: {estado_inicial['cantidad']}")
print(f"Cantidad final: {estado_final['cantidad']}")
print(f"Diferencia: {estado_final['cantidad'] - estado_inicial['cantidad']}")

# Validar números de serie
nuevos_series = set(estado_final['numerosSerie']) - set(estado_inicial['numerosSerie'])
print(f"\nNuevos números de serie agregados: {nuevos_series}")

## 6. Prueba de Baja de Inventario

In [ ]:
def crear_solicitud_salida():
    data_salida = {
        "tipoMovimiento": "SALIDA",
        "inventarioTipo": "INTERIOR",
        "itemId": solicitud_entrada['request']['itemId'],  # Usamos el mismo item
        "cantidad": 2,
        "motivoSolicitud": "Prueba de salida de inventario",
        "numerosSerie": ["TEST001", "TEST002"]  # Tomamos 2 de los números que agregamos
    }
    
    response = requests.post(
        f'{BASE_URL}/inventory-requests',
        headers=headers_auth,
        json=data_salida
    )
    
    print('Respuesta:', json.dumps(response.json(), indent=2))
    return response.json()

solicitud_salida = crear_solicitud_salida()

## 7. Aprobar la solicitud de salida y verificar resultado final

In [ ]:
estado_antes_salida = verificar_item(solicitud_salida['request']['itemId'])
aprobacion_salida = aprobar_solicitud(solicitud_salida['request']['_id'])
estado_final_salida = verificar_item(solicitud_salida['request']['itemId'])

print(f"\nCantidad antes de la salida: {estado_antes_salida['cantidad']}")
print(f"Cantidad después de la salida: {estado_final_salida['cantidad']}")
print(f"Diferencia: {estado_final_salida['cantidad'] - estado_antes_salida['cantidad']}")

# Verificar que los números de serie se removieron
series_removidos = set(estado_antes_salida['numerosSerie']) - set(estado_final_salida['numerosSerie'])
print(f"\nNúmeros de serie removidos: {series_removidos}")